<a href="https://colab.research.google.com/github/galonpy/CS224N-a4/blob/main/movie_recommendations_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Data

In [3]:
PATH = '/content/drive/MyDrive/SIADS-591-data'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

import re
import string
import nltk
nltk.download('wordnet')
from nltk.stem import PorterStemmer


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
def prep_string(text):
    """remove punctuation, whitespace, and make lowercase"""

    # remove whitespace and apply lowercasing
    text_lstrip = str(text).lower().strip()

    # replace puncutation with empty string
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text_lstrip)
    return text

In [7]:
def stem_words(text):
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])

# Data Ingestion

In [8]:
# --- movie lens data ingestion ---

# read movie lens csv files into dataframes
ml_movies = pd.read_csv(f'{PATH}/ml-25m/movies.csv')
ml_ratings = pd.read_csv(f'{PATH}/ml-25m/ratings.csv')
ml_tags = pd.read_csv(f'{PATH}/ml-25m/tags.csv')
# ml_links = pd.read_csv(f'{PATH}/ml-25m/links.csv') # TODO: at this point this data is not useful

# TODO: consider if we want to use these
# ml_gtags = pd.read_csv(f'{PATH}/ml-25m/genome-tags.csv')
# ml_gscores = pd.read_csv(f'{PATH}/ml-25m/genome-scores.csv')

In [9]:
# --- imdb data ingestion ---

# run once to create the imdb.csv file
#imdb_dfs = [pd.read_json(f"./data/part-0{i}.json") for i in range(1, 7)]
# imdb_df = pd.concat(imdb_dfs)
# imdb_df.to_csv("./data/imdb.csv", index=False)

# read imdb csv file into dataframe
# csv created from commented out code above 
#imdb_reviews = pd.read_csv("./data/imdb.csv")

#gabriel alon # TODO: remove for the final copy, but use for quick itterations
imdb_reviews = pd.read_json(f'{PATH}/imdb/part-01.json')

# Data Preprocessing



## imdb data preprocesing

In [10]:
# seperate the movie title and year from the movie column, create two seperate columns for them
imdb_reviews["title"] = imdb_reviews["movie"].apply(lambda x: x.split("(")[0].strip().lower())
imdb_reviews["year"] = imdb_reviews["movie"].apply(
    lambda x: x.split("(")[-1].split(")")[0].strip().replace("–", "")
)

In [11]:
# refactor imdb reviews dataset to the movie level for easy joining with movie lens
imdb_movies = (
    imdb_reviews.groupby(["title", "year"])
    .agg(
        {
            "rating": lambda x: list(x),
            "review_detail": lambda x: list(x),
            "review_summary": lambda x: list(x),
            "helpful": lambda x: list(x),
        }
    )
    .reset_index()
    .rename(
        columns={
            "title": "title",
            "year": "year",
            "rating": "imdb_ratings",
            "review_detail": "imdb_review_detail",
            "review_summary": "imdb_review_summary",
            "helpful": "imdb_helpful",
        }
    )
)


In [12]:
imdb_movies.head(2)

,title,year,imdb_ratings,imdb_review_detail,imdb_review_summary,imdb_helpful
0,,1991,[nan],"[While not without it's flaws, this short is s...",[just trying to recapture a little of the glor...,"[[2, 3]]"
1,,2004,[8.0],[While I have seen some applicable commentary ...,[The reviewers seem a bit homophobic],"[[0, 1]]"


In [13]:
imdb_movies['imdb_review_detail'].iloc[0]

["While not without it's flaws, this short is still one of the most entertaining Looney Tunes shorts in recent years. I've resigned myself to the fact that the golden age of Looney Tunes is long in the past and it'll never get as good as it was back then again in the future. But I still enjoyed this cartoon that plays out like a blooper reel taking place on Bugs Bunny's fifty first and a half birthday special. Even if the Looney Tune's gang acted a tad off-kilter. This cartoon can be found as an extra on disk 1 of the 'Looney Tunes Golden Collection' and has optional commentary.My Grade: C+"]

## movie lens data preprocessing

In [14]:
# seperate the movie title and year from the movie column, create two seperate columns for them
# note: don't run this multiple times during same session # TODO: avoid hard coding here
ml_movies['year'] = ml_movies['title'].apply(lambda x: x[-5:-1])
ml_movies['title'] = ml_movies['title'].apply(lambda x: x[:-6].strip().lower())

In [15]:
# split genres into a list for one-hot encoding
ml_movies['genres'] = ml_movies['genres'].str.split('|')

In [16]:
# # apply one-hot-encoding transformation to list of genres # todo: cut this code if no longer applying one-hot-encoding
# genre_dummies = pd.get_dummies(ml_movies["genres"].apply(pd.Series))
# ml_movies = pd.concat([ml_movies, genre_dummies], axis=1)
# # ml_movies = ml_movies.drop(['genres'], axis=1) #todo: drop after sanity check

In [17]:
# add links to imdb to ml_movies 
# ml_movies = ml_movies.merge(ml_links, on='movieId', how='left') # TODO: this doesn't add value without an id field from idmb dataset

In [18]:
# cleanup movie tags and gather all tags per movie for consistency

# remove punctuation and whitespace from tags
ml_tags["tag"] = ml_tags["tag"].apply(lambda x: prep_string(x))

# replace tags with their stemmed versions for comparison
ml_tags["tag"]= ml_tags["tag"].apply(lambda x: stem_words(x))

# group tags by movie and create a list of tags for each movie to be appended to ml_movies
ml_tags_by_movie = ml_tags.groupby('movieId')['tag'].apply(set).apply(list)
ml_tags_by_movie = pd.DataFrame(ml_tags_by_movie).reset_index()
# ml_tags.groupby('movieId')['tag'].apply(list) # would contain duplicates
# #Todo: ^consider if we want to add weight to the count of each tag. If many people tag a movie the same weight, that may rank higher than a movie with fewer tags but more unique tags

ml_movies = ml_movies.merge(ml_tags_by_movie, on='movieId', how='left')
# ml_tags_by_movie.mege()

In [19]:
# add aggregated movie lens rating information to DataFrame
ratings_agg = (
    (
        ml_ratings.groupby("movieId")
        .agg({"rating": ["mean", "count", "median", "std"]})
        .reset_index()
    )
    .droplevel(level=0, axis=1)
    .rename(
        columns={
            "": "movieId",
            "mean": "rating_mean",
            "count": "rating_count",
            "median": "rating_median",
            "std": "rating_std",
        }
    )
)

ml_movies = ml_movies.merge(ratings_agg, on="movieId", how="left")

In [20]:
ml_movies.head(1)

,movieId,title,genres,year,tag,rating_mean,rating_count,rating_median,rating_std
0,1,toy story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,"[favorit, cartoon, boy, light, tumey to see ag...",3.893708,57309.0,4.0,0.921552


In [21]:
ml_movies['tag'].iloc[:8]

0    [favorit, cartoon, boy, light, tumey to see ag...
1    [monkey, adapt of book, new home, giant insect...
2    [sequel, grun run, jack lemmon, comedinha de v...
3    [clv, chick flick, reveng, singl mother, inter...
4    [sequel, comedi, contracept, midlif crisi, tou...
5    [val kilmer, murder, philosophi, cant rememb, ...
6    [ugli duckl, remadeassabrina1995, romanc, brea...
7    [base on a book, disney, librari system, adapt...
Name: tag, dtype: object

# Joining movie lens and imdb

In [22]:
ml_movies.merge(imdb_reviews, on=["title", "year"], how="inner").to_csv(
    "ml_movies_imdb_joined.csv", index=False
) # todo: how to save to drive

# Exploring the Merged Dataset

In [ ]:
import altair as alt
alt.themes.enable('fivethirtyeight')

# This defaults to `rating_mean` which is the movielens column, we should instead be calling `rating` from IMDB
def scatter_plot_movies_rating(df=ml_movies, rating_col='rating_mean', rating_counts_col='rating_count', top=50):
  mdf = df.copy()
  mdf['genres'] = mdf['genres'].apply(lambda x: x[0] if isinstance(x, list) else '')
  mdf = mdf.groupby(['title', 'year', 'movieId', 'genres']).agg({ rating_counts_col: 'sum', rating_col: 'mean' }).sort_values([rating_counts_col, rating_col], ascending=False).head(top)

  return alt.Chart(mdf.reset_index()).mark_circle(size=60).encode(
        x=alt.X(f'{rating_col}:Q', title='Rating'),
        y=alt.Y('title:N', title='Title'),
        tooltip=['title:N', 'year:O' ,f'{rating_col}:Q'],
        color='genres:N'
    ).configure_axis(grid=False).configure_view(
        width=256,
        height=1024
    ).properties(
        title={
          "text": f"Top {top} Movies", 
          "subtitle": ["Sorted by the total number and average ratings.", "Highlighting the marks will show more information."]
        }
    )

scatter_plot_movies_rating()

In [ ]:
###
# 
# Filter function - filters a dataframe based off the parameters.
#
# df - The dataframe to perform the filter functionality
# filter - The filter string
###
def filter_func(df, filter):
  df = df.copy()

  # Conversion - because lists are unhashable, I can'd do a concat/drop duplicates to return the filtered out items
  # Add this back if we want to pd.concat(new_df, old_df).drop_duplicates()
  #df['genres'] = df['genres'].apply(tuple)
  df['tag'] = df['tag'].apply(lambda d: ' '.join(d) if isinstance(d, list) else d)

  filter = stem_words(filter)

  df['tag'] = df['tag'].astype(str)

  new_df = df[~df['tag'].str.contains(str(filter))]

  return new_df

def query_func(df, query):
  df = df.copy()

  query = stem_words(query)
  df['tag'] = df['tag'].apply(lambda d: ' '.join(d) if isinstance(d, list) else '')

  new_df = df[df['title'].str.contains(str(query)) | df['tag'].str.contains(str(query))]
      
  return new_df

pd.options.display.max_colwidth = 50
  
display(query_func(ml_movies,'cult')['tag'])

In [ ]:
df = filter_func(ml_movies, 'abuse')

In [ ]:
exp3 = filter_func(query_func(ml_movies, 'cult'), 'dumb down')
exp3.head(3)

# Posters

In [ ]:
ml_movies_clean = ml_movies.copy()
print(len(ml_movies_clean))

In [ ]:
def test_apply(x):
    try:
        return int(x)
    except ValueError:
        return None
ml_movies_clean['year'] = ml_movies_clean['year'].apply(test_apply).dropna()

In [ ]:
images = pd.read_csv('/content/drive/MyDrive/SIADS-591-data/duplicate_free_41K.csv')
images = images[['poster','title','year']]
print(len(images))
images['title'] = images['title'].apply(lambda x: prep_string(x))
poster_and_prior =pd.merge(ml_movies_clean,images, on=['title','year'])
print(len(poster_and_prior))


In [ ]:
from PIL import Image
from io import BytesIO
import os
import urllib.request
import json
import altair as alt
import pandas as pd
import base64, io, IPython
import requests

In [ ]:
def prepare_images(poster_sample,n):
  #poster_sample is the links to the posters of the movie titles we want to visualize
  show_img = []
  san_img = []
  count = 0
  png_names = []
  for i in poster_sample:
    name = f"/content/drive/MyDrive/SIADS-591-data/local-filename{str(count)}.jpg"
    ex = urllib.request.urlretrieve(i, name)#"/content/drive/MyDrive/SIADS-591-data/local-filename{i}.jpg")
    im = Image.open(name)
    png_name = f"/content/drive/MyDrive/SIADS-591-data/local-filename{str(count)}.png"
    png_names.append(png_name)
    im.save(png_name)
    count +=1
  imgCode = []
  images  = png_names[:n]
  for imgPath in images:
      image = Image.open(imgPath) #PilImage
      output = io.BytesIO()    
      image.save(output, format='JPEG')
      encoded_string = "data:image/jpeg;base64,"+base64.b64encode(output.getvalue()).decode()
      imgCode.append(encoded_string)
  print(len(imgCode))
  return imgCode

In [ ]:
def poster_visual_altair(poster_sample, n,row_level):
  #pass in n number of images to show in that row
  imgCode = prepare_images(poster_sample,n)
  x = [i for i in range(1,50000,6000)]
  x = x[:n]
  y = [row_level]*n

  source = pd.DataFrame({"x": x, "y": y, "img": imgCode})
  vis = alt.Chart(source).mark_image(size=5,
      width=250,
      height=250
  ).encode(
      x=alt.X('x', axis=None), #axis=alt.Axis(labels=False,grid=False,title='')),
      y=alt.Y('y',axis=None), #axis=alt.Axis(labels=False,grid=False, title='')),
      url='img')
  return vis

In [ ]:
poster_and_prior.head(1)

In [ ]:
#given lists of titles and years we can make a dataframe to select the poster links
selected_titles = ["1408","2012","50/50"]
selected_years = [2007,2009,2011]
data_tuples = list(zip(selected_titles,selected_years))
selected_df = pd.DataFrame(data_tuples, columns=['title','year'])
selected_final = pd.merge(poster_and_prior,selected_df,on=['title','year'])
print(len(selected_final))
selected_final = selected_final.drop_duplicates(
  subset = ['title', 'year'],
  keep = 'last').reset_index(drop = True)
selected_posters = selected_final['poster']

# Input Widget

This widget input field filters a pandas dataframe of movies based off the movie title. 

The entered string is compared against the 'title' column of the dataframe to display only those movies whose title contains the string. 

Ensure that the input is a valid string, otherwise the filtering may not work as expected. 

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output

query_input =widgets.Text(
    value='',
    placeholder='Search for a movie by title',
    description='Search',
    disabled=False   
)

filter_input = widgets.Text(
    value='',
    placeholder='Filter out movies using keywords',
    description='Filter',
    disabled=False   
)


def query_filter(button):
  filter = filter_input.value
  query = query_input.value
  df = filter_func(query_func(ml_movies, query), filter)
  clear_output()
  display(df)

submit_button = widgets.Button(description='submit')

submit_button.on_click(query_filter)

widgets.VBox([query_input, filter_input, submit_button])

# Combining Input Widget with Poster

In [ ]:
ml_movies[ml_movies['title'] == 'last holiday']

In [ ]:
%%capture
!pip install altair_viewer

In [ ]:
def query_filter(button):
  if not filter_input.value:
    filter = 'None Marked'
  else:
    filter = filter_input.value
  query = query_input.value
  df = filter_func(query_func(ml_movies, query), filter)
  #gabriel alon
  title = df['title'].tolist()[:3]
  year = df['year'].tolist()[:3]
  year = [float(int(i)) for i in year]
  data_tuples = list(zip(title,year))
  selected_df = pd.DataFrame(data_tuples, columns=['title','year'])
  selected_final = pd.merge(poster_and_prior,selected_df,on=['title','year'])
  selected_posters = selected_final['poster']
  chart = poster_visual_altair(selected_posters, len(selected_posters),1)
  clear_output() #this removes the search and filter buttons though
  display(chart)
submit_button = widgets.Button(description='submit')

submit_button.on_click(query_filter)

widgets.VBox([query_input, filter_input, submit_button])

#ie search: last holiday

#single word in title doesn't work
#doesnt seem that tags work or are searched
#no warning that image isnt available but movie is